In [4]:
import math
import numpy as np
import tensorflow as tf
import cv2 as cv
import os
from feature_extraction import *

IMG_WIDTH = 64
IMG_HEIGHT = 32

In [5]:
class eqn:
    def __init__(self, y):
        self.y = y
    
    def eval(self, x):
        return (self.y[0] ** x) + (self.y[1] * x) + self.y[2]

def flatten(x):
    final = []
    for pos1 in range(len(x)):
        for pos2 in range(len(x[0])):
            for pos3 in range(len(x[0][0])):
                final.append(x[pos1][pos2][pos3])
    return final

In [6]:
features = []
for path in os.listdir(os.getcwd() + "\\Sidewalks"):
    img = cv.imread(os.getcwd() + "\\Sidewalks\\" + path)
    features.append(flatten([edge_detection(img, IMG_WIDTH, IMG_HEIGHT), hue_detection(img, IMG_WIDTH, IMG_HEIGHT)]))

train = np.array(features)

In [7]:
def custom_loss(features, lines):
    losses = []
    for pos in range(0, len(features)):
        x = features.numpy()[pos]
        y = lines.numpy()[pos]
        
        SD = 10
        HEIGHT = 40
        normal = lambda SD, HEIGHT, M, x: (1/(SD*math.sqrt(2*math.pi))) * ((math.e ** ((-1/2) * (((x-M) / SD) ** 2)))) * HEIGHT

        line = eqn(y)
        mid = int(len(x) / 2)

        max = 0
        loss1 = 0
        loss2 = 0
        for x_pos in range(0, IMG_WIDTH):
            y_line_pos = line.eval(x_pos)
            for y_pos in range(0, IMG_HEIGHT):
                val = normal(SD, HEIGHT, y_line_pos, y_pos)
                pos = int((IMG_WIDTH * y_pos) + x_pos)
                loss1 += x[pos] * val / 255
                loss2 += x[pos + mid] * val / 255
                max += val
        if max != 0:
            losses.append(min(((max - loss1) / max), ((max - loss2) / max)))
        else:
            losses.append(1)
    return tf.convert_to_tensor(losses)

In [8]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(5, kernel_initializer='RandomNormal', activation='relu'),
    tf.keras.layers.Dense(3)
    ])

In [9]:
model = create_model()
model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'], run_eagerly=True)
model.fit(x=train, 
          y=train, 
          epochs=20, 
          validation_data=(train, train)
)



Epoch 1/20


C:\Users\kahns_\AppData\Local\Temp\__autograph_generated_file5h2glowd.py:24: RuntimeWarning: overflow encountered in scalar power
  normal = ag__.autograph_artifact(lambda SD, HEIGHT, M, x: 1 / (ag__.ld(SD) * ag__.converted_call(ag__.ld(math).sqrt, (2 * ag__.ld(math).pi,), None, fscope)) * ag__.ld(math).e ** (-1 / 2 * ((ag__.ld(x) - ag__.ld(M)) / ag__.ld(SD)) ** 2) * ag__.ld(HEIGHT))


ValueError: No gradients provided for any variable: (['dense/kernel:0', 'dense/bias:0', 'dense_1/kernel:0', 'dense_1/bias:0', 'dense_2/kernel:0', 'dense_2/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense/kernel:0' shape=(4096, 4096) dtype=float32, numpy=
array([[-0.00670535, -0.00027026,  0.01395421, ...,  0.01834154,
         0.01370252, -0.00651409],
       [-0.0123529 ,  0.00171422, -0.01640284, ..., -0.00574278,
         0.00348815,  0.00272698],
       [-0.01812827, -0.02528524,  0.00066675, ...,  0.02652769,
         0.01752108,  0.02026949],
       ...,
       [ 0.02385384, -0.0012596 , -0.00487465, ...,  0.00410222,
         0.00269611,  0.01718312],
       [-0.01750427,  0.02134139,  0.01771515, ..., -0.02160138,
         0.01715901,  0.00105419],
       [ 0.0018922 ,  0.00220445, -0.01667919, ..., -0.01324837,
         0.0192206 , -0.02455427]], dtype=float32)>), (None, <tf.Variable 'dense/bias:0' shape=(4096,) dtype=float32, numpy=array([0., 0., 0., ..., 